In [1]:
import pathlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
from orchid import (project_loader as opl)

In [4]:
import toolz.curried as toolz

In [5]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [6]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [7]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

{'permian-a': WindowsPath('c:/src/Orchid.IntegrationTestData/Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac'),
 'gng': WindowsPath('c:/src/Orchid.IntegrationTestData/GnG_DemoProject_wDataFrames.ifrac')}

In [8]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

{'permian-a': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x000001EDA1067820>,
 'gng': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x000001EDA1067880>}

In [9]:
def make_project_data_frames_by_id(project):
    return {df.ObjectId: df for df in project.DataFrames.Items}

project_net_data_frames_by_id = toolz.pipe(
    projects,
    toolz.valmap(make_project_data_frames_by_id),
)
project_net_data_frames_by_id

{'permian-a': {<System.Guid object at 0x000001EDA1067F40>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x000001EDA1067DF0>,
  <System.Guid object at 0x000001EDA106D1C0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x000001EDA106D190>,
  <System.Guid object at 0x000001EDA106D3D0>: <Orchid.FractureDiagnostics.Factories.DataFrames.MicroseismicDataFrame object at 0x000001EDA106D3A0>},
 'gng': {<System.Guid object at 0x000001EDA106D460>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x000001EDA106D400>,
  <System.Guid object at 0x000001EDA106D6A0>: <Orchid.FractureDiagnostics.Factories.DataFrames.FaultTraceSetDataFrame object at 0x000001EDA106D670>,
  <System.Guid object at 0x000001EDA106D8E0>: <Orchid.FractureDiagnostics.Factories.DataFrames.StageDataFrame object at 0x000001EDA106D8B0>,
  <System.Guid object at 0x000001EDA106DB20>: <Orchid.FractureDiagnostics.Factories.DataFrames.WellLogSet

In [10]:
def make_project_data_frames_by_name(project):
    return {df.Name: df for df in project.DataFrames.Items}

project_net_data_frames_by_name = toolz.pipe(
    projects,
    toolz.valmap(make_project_data_frames_by_name),
)
project_net_data_frames_by_name

{'permian-a': {'Project Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x000001EDA104DA60>,
  'FDI Observations': <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x000001EDA1072070>,
  'C3-Microseismic Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.MicroseismicDataFrame object at 0x000001EDA10720A0>},
 'gng': {'Project Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x000001EDA10720D0>,
  'Fault Trace Set Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.FaultTraceSetDataFrame object at 0x000001EDA1072100>,
  'Stage Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.StageDataFrame object at 0x000001EDA1072130>,
  'Well Log Set Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.WellLogSetDataFrame object at 0x000001EDA1072190>,
  'Horizon Marker Set Data Frame01': <Orchid.FractureDiagnostics.Factories.Dat

In [11]:
len(list(toolz.get('permian-a', project_net_data_frames_by_name)))

3

In [12]:
len(list(toolz.get('gng', project_net_data_frames_by_name)))

5

In [98]:
def extract_net_data_frame_identities(df):
    return {
        'object_id': str(df.ObjectId),
        'name': df.Name,
        'display_name': df.DisplayName,
    }

@toolz.curry
def build_net_data_frame_identities(ndfs):
    return toolz.map(extract_net_data_frame_identities, ndfs)
    
net_data_frame_identities = toolz.pipe(
    project_net_data_frames_by_name,
    toolz.valmap(lambda dfs_by_name: dfs_by_name.values()),
    toolz.valmap(build_net_data_frame_identities),
    toolz.valmap(list),
)
net_data_frame_identities

{'permian-a': [{'object_id': '0d2ec4b2-5766-461a-b57d-cc711576f46f',
   'name': 'Project Data Frame 01',
   'display_name': None},
  {'object_id': '0339e49a-a534-4d6f-b218-9862eeb73019',
   'name': 'FDI Observations',
   'display_name': None},
  {'object_id': 'a48e71d9-50ce-49c4-bde2-c1915cb87bd4',
   'name': 'C3-Microseismic Data Frame 01',
   'display_name': None}],
 'gng': [{'object_id': '9acfc88f-44f3-4f23-ac78-6ca94bae2d84',
   'name': 'Project Data Frame 01',
   'display_name': None},
  {'object_id': 'db5dc0c7-9132-4270-9bff-2bbf32ed93e3',
   'name': 'Fault Trace Set Data Frame 01',
   'display_name': None},
  {'object_id': 'e16f00ad-4c42-4726-8996-22a4632beaa9',
   'name': 'Stage Data Frame 01',
   'display_name': None},
  {'object_id': 'a287e63d-fd15-48e8-b5fc-99eb57244f18',
   'name': 'Well Log Set Data Frame 01',
   'display_name': None},
  {'object_id': '1cef6417-acb4-478c-b270-ca7022fc6003',
   'name': 'Horizon Marker Set Data Frame01',
   'display_name': None}]}

In [128]:
def make_data_frame_identities_summary(item):
    project_name, ndf_id_sums = item
    summaries = toolz.pipe(
        ndf_id_sums,
        toolz.map(lambda ndf_id_sum: toolz.merge(ndf_id_sum, {'project': project_name})),
    )
    return project_name, summaries

data_frame_identities_summary = toolz.itemmap(make_data_frame_identities_summary, net_data_frame_identities)
data_frame_identities_summary

{'permian-a': <map at 0x1ed9f464d90>, 'gng': <map at 0x1ed9f457a90>}

In [129]:
pd.DataFrame(data=data_frame_identities_summary['permian-a'])

,object_id,name,display_name,project
0,0d2ec4b2-5766-461a-b57d-cc711576f46f,Project Data Frame 01,None,permian-a
1,0339e49a-a534-4d6f-b218-9862eeb73019,FDI Observations,None,permian-a
2,a48e71d9-50ce-49c4-bde2-c1915cb87bd4,C3-Microseismic Data Frame 01,None,permian-a


In [130]:
pd.DataFrame(data=data_frame_identities_summary['gng'])

,object_id,name,display_name,project
0,9acfc88f-44f3-4f23-ac78-6ca94bae2d84,Project Data Frame 01,None,gng
1,db5dc0c7-9132-4270-9bff-2bbf32ed93e3,Fault Trace Set Data Frame 01,None,gng
2,e16f00ad-4c42-4726-8996-22a4632beaa9,Stage Data Frame 01,None,gng
3,a287e63d-fd15-48e8-b5fc-99eb57244f18,Well Log Set Data Frame 01,None,gng
4,1cef6417-acb4-478c-b270-ca7022fc6003,Horizon Marker Set Data Frame01,None,gng
